In [0]:
from datetime import datetime
today_str = datetime.now().strftime("%Y-%m-%d")
dbutils.widgets.text("env", "aws")
dbutils.widgets.text("storage_root", "s3://pakeyj-data-sales")
dbutils.widgets.text("process_date", today_str)

env = dbutils.widgets.get("env")
storage_root = dbutils.widgets.get("storage_root")
process_date = dbutils.widgets.get("process_date")

print("env:", dbutils.widgets.get("env"))
print("storage_root:", dbutils.widgets.get("storage_root"))
print("process_date:", dbutils.widgets.get("process_date"))

In [0]:
from src.common.config import load_config

cfg = load_config(env=env, storage_root=storage_root, process_date=process_date)
storage_root = cfg.storage_root
process_date = cfg.process_date
orders_out = cfg.paths["raw_orders_daily"]
items_out  = cfg.paths["raw_order_items_daily"]
meta_out   = f"{storage_root}/raw/_generator_runs"

def peek(df, name, n=10):
    print(f"\n=== {name} ===")
    print(f"rows: {df.count()}")
    df.limit(n).show(truncate=False)

df = spark.read.json(orders_out) 


In [0]:
peek(df, "orders")

In [0]:
from pyspark.sql import functions as F
# bronze_paths = [orders_out,items_out]
def convert_to_delta(df, path):
    
df_today = df.filter(F.col("ingest_date") == process_date)

(df_today
  .write.format("delta")
  .mode("overwrite")
  .option("replaceWhere", f"ingest_date = '{process_date}'")
  .partitionBy("ingest_date")
  .save(orders_out)
)

